In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import copy
from uniform_attachment_model  import matrix_uniform_attachment
from tqdm.notebook import tqdm
import pandas as pd
import time
import copy
import  heapq
from reg_three_g import make_that_0_into_1_with_transpose
from line_tree import create_line_tree_adjacency_matrix
from reg_three_g import final_3_reg_tree
import seaborn as sns
import os
import math
import matplotlib.animation as animation
import imageio.v2 as imageio
import ast

/home/adsayan/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


0.04873085021972656
time take  is 4.5299530029296875e-05
printing time is (5.9604644775390625e-06)


In [2]:
def find_barycenter_prob(matrix, vertex_to_remove):

    matrix_copy = copy.deepcopy(matrix)

    row_c = [0, 1]

    for i in vertex_to_remove:
        if vertex_to_remove.count(i) >= 2:
            return i  

    while len(row_c) != 0:
        row, column = row_colum_returner(matrix_copy, vertex_to_remove)
        row_c = row

        if len(row_c) == 0:
            break

        for r, c in zip(row, column):
            matrix_copy = make_that_1_into_0_with_transpose(matrix_copy, r, c)

 
    for s, i in enumerate(matrix_copy):
        if sum(i) == 3:
            return s  #

    for i in vertex_to_remove:
        if sum(matrix_copy[i]) == 2:
            return i  

 
    return None
def row_colum_returner(matrix, vertex_to_remove):
    s= 0 
    column = []
    row = []
    for i in matrix:
        if sum(i) == 1:
            if s in vertex_to_remove:
                pass
            if s not in vertex_to_remove:
                d = 0
                for v in i:
                    if v==1:
                        row.append(s)
                        column.append(d)
                    d+=1
                        
    
        s+=1

    return row, column

def calculate_u_new(matrix):
    row_count = max(len(i) for i in matrix)
    u_new = [0] * row_count
    u_old = [1 / row_count] * row_count
    
    for i in range(0, row_count):
        for j in range(0, row_count):
            for k in range(0, row_count):
                center = find_barycenter_prob(matrix, vertex_to_remove=[i, j, k])
                u_new[center] += u_old[i] * u_old[j] * u_old[k]
    
    return u_new


def find_barycenter_deterministic(adj_matrix):
    leaf_cut = []    
    adj_matrix_copy = copy.deepcopy(adj_matrix)
    
    n = len(adj_matrix_copy)  # Number of nodes, number of rows
    degrees = np.sum(adj_matrix_copy, axis=1)  # Sum of row = degree
    nodes = set(range(n))  # All vertices
    
    while len(nodes) > 2: 
        # Find all leaf nodes (nodes with degree 1)
        leaf_nodes = [i for i in nodes if degrees[i] == 1]  # Only those with a single 1 are leaves
        
        for leaf in leaf_nodes:
            nodes.remove(leaf)  # Remove the leaf
            degrees[leaf] = 0  # Degree is then 0

            for neighbor in range(n):  # Check all neighbors
                if adj_matrix_copy[leaf][neighbor] == 1:  # If there is a connection
                    degrees[neighbor] -= 1  # Decrement the degree of the neighbor
                    adj_matrix_copy[leaf][neighbor] = 0  # Change the 1 to 0
                    adj_matrix_copy[neighbor][leaf] = 0  # Also change its "transpose position" to 0
        leaf_cut.append(1)

    # At this point, there should only be 1 or 2 nodes left
    remaining_nodes = list(nodes)  # Convert the set to a list

    if len(remaining_nodes) == 1:  # If only 1 vertex remains
        return remaining_nodes, adj_matrix_copy, sum(leaf_cut)
    elif len(remaining_nodes) == 2:  # If 2 vertices remain
        return remaining_nodes, adj_matrix_copy, sum(leaf_cut)

def make_that_1_into_0_with_transpose(matrix, row, column):
    matrix[row][column] = 0
    matrix[column][row] = 0


    return matrix

def draw_graph_from_adjacency_matrix(adjacency_matrix):
    G = nx.Graph()

    for i in range(len(adjacency_matrix)):
        for j in range(len(adjacency_matrix[i])):
            if adjacency_matrix[i][j] != 0: 
                G.add_edge(i, j, weight=adjacency_matrix[i][j])


    G.add_nodes_from(range(len(adjacency_matrix)))

    pos = nx.spring_layout(G, seed=42)

    # Draw the graph
    plt.figure(figsize=(8, 6))
    nx.draw(
        G,
        pos,
        with_labels=True,
        node_color='lightblue',
        edge_color='black',
        node_size=10,
        font_size=10,
        width=0.5
    )
    plt.title("Graph from Adjacency Matrix")
    plt.show()
def calculate_u_new(matrix, u_old = None, centers = None):
    centers = []
    row_count = max(len(i) for i in matrix)
    u_new = [0] * row_count

    if u_old is None:
        u_old = [1 / row_count] * row_count

    if  not centers:
        for i in range(0, row_count):
            for j in range(0, row_count):
                for k in range(0, row_count):
                    center = find_barycenter_prob(matrix, vertex_to_remove=[i, j, k])
                    no_change_center = [i,j,k,center]
                    u_new[center] += u_old[i] * u_old[j] * u_old[k]
                    centers.append(no_change_center)
        return u_new,centers


    if centers:
        for element in centers:
            u_new[element[3]] += u_old[element[0]] * u_old[element[1]] * u_old[element[2]]
            return u_new, centers
            


def entropy_calculator(list_s):
    entropy_sum_list = []
    for i in list_s:
        if i < 0.0000000001:
            value = 0
        else:
            value = i * np.log(i)
            entropy_sum_list.append(value)
    
    return -sum(entropy_sum_list)

def main_entropy_for_each_interation(n, matrix, lower_barrier = np.log(2).round(10)):
    start_time = time.time() 
    total_steps = n
    progress_bar = tqdm(total=total_steps, desc="Computing", dynamic_ncols=True)

    list_with_all_iteration = []
    u_old = None
    centers = None
    for s in range(n):
        elapsed_time = time.time() - start_time
        progress_bar.set_description(f"Elapsed: {elapsed_time:.2f} sec")
        u_calculated, los_centers = calculate_u_new(matrix, u_old, centers)
        entropy_received = (entropy_calculator(u_calculated)).round(4)
        centers = los_centers
        u_old = u_calculated

        dict_2_send = {
            "interation": s,
            "entropy": entropy_received,
            'u_calculated': u_calculated
        }
        list_with_all_iteration.append(dict_2_send)

        if entropy_received < lower_barrier:  
            progress_bar.n = total_steps  # Set progress to max
            progress_bar.refresh()  # Force update the UI
            progress_bar.close
            break  

        progress_bar.update(1)  

    progress_bar.close()  
    return list_with_all_iteration
    
def runner(iterations, node_size):
    
    node_counter = []
    edge_counter = []
    
    edge_count = 0
    vertex_count = 0
    for i in set(range(iterations)):
        matrix = matrix_uniform_attachment(node_size)
        sd ,ad, cuts = find_barycenter_deterministic(matrix)


        
        if len(sd) == 2:#it is an edge
            edge_count+=1
            edge_counter.append(sd[0])
            edge_counter.append(sd[1])
        elif len(sd) == 1: #is is an vertex
            vertex_count+=1
            node_counter.append(sd[0])
    return edge_count, vertex_count, node_counter, edge_counter
def generate_galton_watson_adjacency(n, p, max_nodes=100, max_iterations=20):
    """Generate the adjacency matrix for a Galton-Watson process."""
    matrix = [[0]]  
    rows_completed = []  
    
    iteration = 0
    while iteration < max_iterations:
        new_nodes_added = False 

        for i in range(len(matrix)):
            if i in rows_completed:
                continue

            locally_completed_int = []
            how_many_kids = quick_binomial(n, p)

            if how_many_kids > 0:
                new_nodes_added = True  

            for _ in range(how_many_kids):
                while len(matrix[i]) <= max_finder_in_list(locally_completed_int):
                    matrix[i].append(0)
                
                matrix[i].append(1)
                locally_completed_int.append(len(matrix[i]) - 1)

            how_many_rows_required = len(matrix[i])
            while len(matrix) < how_many_rows_required:
                matrix.append([0] * how_many_rows_required)

            for row in matrix:
                while len(row) < len(matrix):
                    row.append(0)

            for child_index in locally_completed_int:
                make_that_0_into_1_with_transpose(matrix, i, child_index)

            rows_completed.append(i)

        iteration += 1

        if not new_nodes_added:  
            break

    return matrix


In [3]:

def make_that_1_into_0_with_transpos(matrix, row, column):
    matrix[row][column] = 0
    matrix[column][row] = 0

    
def merge_trees(matrix_1, matrix_2, random_vertex_selection = True, vertex_from_matrix_1 = None, vertex_from_matrix_2 = None):
    matrix_1_cp = copy.deepcopy(matrix_1)
    matrix_2_cp = copy.deepcopy(matrix_2)


    for i in range(len(matrix_2_cp)):
        for li in matrix_1_cp:
            li.append(0)


    for i in range(len(matrix_1_cp)):
        for li in matrix_2_cp:
            li.insert(0, 0)
        


    for row in matrix_2_cp:
        matrix_1_cp.append(row)

    if random_vertex_selection: 
        vertex_for_matrix_1 = np.random.choice(list(range(len(matrix_1))))
        vertex_for_matrix_2_og = np.random.choice(list(range(len(matrix_2))))
        vertex_2_to_change = vertex_for_matrix_2_og + len(matrix_1)
        print(f"vertex {vertex_for_matrix_1} from matrix_1 and {vertex_for_matrix_2_og} from matrix_2_were chosen")
        matrix_final = make_that_0_into_1_with_transpose(matrix_1_cp, vertex_for_matrix_1, vertex_2_to_change)
        return matrix_final, vertex_for_matrix_1, vertex_for_matrix_2_og

    else:
        if vertex_from_matrix_1 is None:
            vertex_for_matrix_1 = np.random.choice(list(range(len(matrix_1))))
            vertex_2_to_change = vertex_from_matrix_2 + len(matrix_1)
            print(f"vertex {vertex_for_matrix_1} from matrix_1 and {vertex_from_matrix_2} from matrix_2_were chosen")
            matrix_final = make_that_0_into_1_with_transpose(matrix_1_cp, vertex_for_matrix_1, vertex_2_to_change)
            return matrix_final, vertex_for_matrix_1, vertex_from_matrix_2
            
        else:
            vertex_for_matrix_1 = vertex_from_matrix_1
            vertex_2_to_change = vertex_from_matrix_2 + len(matrix_1)
            print(f"vertex {vertex_for_matrix_1} from matrix_1 and {vertex_from_matrix_2} from matrix_2_were chosen")
            matrix_final = make_that_0_into_1_with_transpose(matrix_1_cp, vertex_for_matrix_1, vertex_2_to_change)
            return matrix_final, vertex_for_matrix_1, vertex_from_matrix_2



In [4]:
def diffusion_process(M, delta):
    matrix = copy.deepcopy(M)
    size = np.array(matrix).shape[0]
    m = np.identity(size)
    first_half = (1-delta) * m
    second_half = return_deg(M, delta)
    return (first_half + second_half)
    

def return_deg(matrix, delta):
    ma = copy.deepcopy(matrix)
    clean = copy.deepcopy(matrix)
    for i in ma:
        deg = sum(i)
        for v, num in enumerate(i):
            i[v] = num / deg
    ma = np.matrix.transpose(np.array(ma))
    return (delta *ma) 

In [12]:
centers = None
u_old = None
matrix = matrix_uniform_attachment(15)
delta = 0.3
matri = diffusion_process(matrix, delta)
iterations_done = 0
last_four_value = []

tolerance = 1e-5
for _ in range(20):
    x,y = calculate_u_new(matrix, u_old = u_old, centers = centers)
    centers = y
    value = np.matmul(matri, np.array(x))
    u_old = value
    entropy = entropy_calculator(value)

    if len(last_four_value) < 4:
        last_four_value.append(entropy)
    elif len(last_four_value) >= 4:
        last_four_value.remove(last_four_value[0])
        last_four_value.append(entropy)


    iterations_done+=1
    if len(last_four_value) >= 4:
        diffs = [np.linalg.norm(last_four_value[i] - last_four_value[i - 1]) for i in range(1, 4)]
        if np.mean(diffs) < tolerance:
            break



        

KeyboardInterrupt: 

In [26]:
def main_guy(delta,model_type, matrix_size=15, tolerance=1e-5, max_tolerance = 40):
    """for model type, put either random_uni, straight_line, tree_3, note that for tree_3 it's iterations done"""
    centers = None
    u_old = None
    last_before_u = None

    if model_type == "random_uni":
        matrix = matrix_uniform_attachment(matrix_size)

    elif model_type == "straight_line":
        matrix = create_line_tree_adjacency_matrix(matrix_size)
    elif model_type == "tree_3":
        matrix = final_3_reg_tree(matrix_size)
    matri = diffusion_process(matrix, delta)
    iterations_done = 0
    last_four_value = []

    start_time = time.time()
    pbar = tqdm(total=max_tolerance, desc="Processing Iterations", unit="step") 

    for _ in range(max_tolerance):
        x, y = calculate_u_new(matrix, u_old=u_old, centers=centers)
        centers = y
        value = np.matmul(matri, np.array(x))

        u_old = value
        entropy = entropy_calculator(value)


        if entropy < 0.1:
            pbar.n = pbar.total
            pbar.update(0)
            break
            
        

        if len(last_four_value) < 4:
            last_four_value.append(entropy)
        else:
            last_four_value.pop(0)  
            last_four_value.append(entropy)

        iterations_done += 1

        elapsed_time = time.time() - start_time
        pbar.set_description(f"Processing Iterations | Time: {elapsed_time:.2f}s")
        pbar.update(1)  

        if len(last_four_value) >= 4:
            diffs = [np.linalg.norm(last_four_value[i] - last_four_value[i - 1]) for i in range(1, 4)]
            if np.mean(diffs) < tolerance:
                pbar.n = pbar.total
                pbar.update(0)
                break

    pbar.close()


    return u_old, iterations_done, np.mean(last_four_value)

def make_gif(df):
    os.makedirs('frames', exist_ok=True)
    folder_path = "gifs"
    os.makedirs(folder_path, exist_ok=True)
    
    filenames = []

    for i in range(len(df)):
        frame_path = f'frames/frame_{i:03d}.png'
        filenames.append(frame_path)
        
        fig, ax = plt.subplots(figsize=(14, 12))
        
        vertex = df.iloc[i]['vertex']
        u_old = df.iloc[i]['u_old']
        u_old = [i/sum(u_old) for i in u_old]
        
        ax.bar(range(vertex), u_old)
        ax.set_xlabel('vertex')
        ax.set_ylabel('Prob')
        ax.set_title(f"Prob distribution vs. Delta, Vertex count is {df['vertex'][0]}, "
                     f"Initial entropy: {round(np.log(df['vertex'][0]), 3)}")

        ax.set_ylim(0, max(u_old) + 1)
        ticks = np.arange(0, max(u_old) + 0.01, 1)
        ax.set_yticks(ticks)
        ax.legend([f"delta is {df.delta[i]}"])
        
        fig.savefig(frame_path)
        plt.close(fig)
    
    filename = f"Prob distribution vs. Delta, Vertex count is {df['vertex'][0]}, and initial entropy is {round(np.log(df['vertex'][0]), 3)}"
    gif_filename = f"{folder_path}/{filename}.gif"
    
    with imageio.get_writer(gif_filename, mode='I', duration=4.0, loop = 0) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)
    

    for filename in filenames:
        os.remove(filename)
    
    print(f"GIF saved as '{gif_filename}'")

In [57]:
delta_range = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
matrix_sizes = list(range(10, 11))
matrix_sizes_2 =list(range(15, 16))
def boss_runner(delta_range, matrix_sizes, tolerance = 1e-3, model_type = "straight_line"):

    all_data = []
    
    for matrix_size in matrix_sizes:   
        iteration = []
        for delta in delta_range:
            for _ in range(10):
                local_it = []
                local_entropy = []
                u_new, iterr, entropy = main_guy(delta, model_type=model_type, matrix_size=matrix_size, tolerance=tolerance)
                local_it.append(iterr)
                local_entropy.append(entropy)
            iteration.append({'delta':delta ,
                              'vertex':matrix_size,
                            'iterration':np.mean(local_it),
                            'entropy': np.mean(local_entropy),
                             'u_old' : u_new})
    
        all_data.append(iteration)
        df = pd.DataFrame(iteration)
        test_df = df
        make_gif(df)

    
    return all_data, df

In [58]:
_, df15 = boss_runner(delta_range, matrix_sizes)
_, df10 =  boss_runner(delta_range, matrix_sizes_2)

Processing Iterations:   0%|          | 0/40 [00:00<?, ?step/s]

Processing Iterations:   0%|          | 0/40 [00:00<?, ?step/s]

Processing Iterations:   0%|          | 0/40 [00:00<?, ?step/s]

Processing Iterations:   0%|          | 0/40 [00:00<?, ?step/s]

Processing Iterations:   0%|          | 0/40 [00:00<?, ?step/s]

...

Processing Iterations:   0%|          | 0/40 [00:00<?, ?step/s]

Processing Iterations:   0%|          | 0/40 [00:00<?, ?step/s]

Processing Iterations:   0%|          | 0/40 [00:00<?, ?step/s]

Processing Iterations:   0%|          | 0/40 [00:00<?, ?step/s]

GIF saved as 'gifs/Prob distribution vs. Delta, Vertex count is 15, and initial entropy is 2.708.gif'


In [59]:
make_superpose_df(df10, df15)

GIF saved as gifs/superposition_between_10_and_15.gif


/tmp/ipykernel_210476/3107715350.py:53: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [60]:
df10

,delta,vertex,iterration,entropy,u_old
0,0.1,15,15.0,0.447755,"[4.3403476444411627e-125, 8.42553849632893e-65..."
1,0.2,15,18.0,0.813146,"[7.792638734466412e-90, 3.361330109508181e-45,..."
2,0.3,15,17.0,1.114451,"[3.4216376921520563e-66, 1.9925315146568837e-3..."
3,0.4,15,15.0,1.307180,"[5.272477121519376e-52, 2.164335563662623e-26,..."
4,0.5,15,14.0,1.430576,"[6.806619352205436e-43, 7.214867537388309e-22,..."
5,0.6,15,13.0,1.523660,"[3.764637764620583e-36, 1.4223727838760121e-18..."
6,0.7,15,13.0,1.600060,"[2.8483868292798114e-31, 4.1563477954619454e-1..."
7,0.8,15,13.0,1.665968,"[2.38462069982021e-27, 3.856336121802722e-14, ..."
8,0.9,15,12.0,1.724701,"[6.296669113263092e-24, 1.694321961651188e-12,..."


In [47]:
def make_superpose_df(df1, df2):
    os.makedirs('frames', exist_ok=True)
    folder_path = "gifs"
    os.makedirs(folder_path, exist_ok=True)

    filenames = []

    for i in range(len(df1)):
        frame_path = f'frames/frame_{i:03d}.png'
        filenames.append(frame_path)

        fig, ax = plt.subplots(figsize=(14, 6))

        vertex1 = df1.iloc[i]['vertex']
        u_old1 = df1.iloc[i]['u_old']
        u_old1 = [v / sum(u_old1) for v in u_old1]

        vertex2 = df2.iloc[i]['vertex']
        u_old2 = df2.iloc[i]['u_old']
        u_old2 = [v / sum(u_old2) for v in u_old2]

        max_vertex = max(vertex1, vertex2)
        common_range = max_vertex + 2
        x_common = np.arange(common_range)

        center1 = common_range // 2 - vertex1 // 2
        center2 = common_range // 2 - vertex2 // 2

        x1 = np.arange(vertex1) + center1
        x2 = np.arange(vertex2) + center2

        ax.bar(x1, u_old1, width=0.4, color='blue', alpha=0.7, label='Tree 1')
        ax.bar(x2, u_old2, width=0.4, edgecolor='red', fill=False, linestyle='--', linewidth=2, label='Tree 2')


        ax.set_xlabel('Vertex')
        ax.set_ylabel('Probability')
        ax.set_title(f"Superposition of Tree 1 and Tree 2 at Frame {i}")
        ax.set_ylim(0, 1)

        ax.legend()
        ax.grid(True, linestyle='--', alpha=0.3)

        fig.tight_layout()
        fig.savefig(frame_path)
        plt.close(fig)

    filename = f"superposition_between_10_and_15"
    gif_filename = f"{folder_path}/{filename}.gif"

    with imageio.get_writer(gif_filename, mode='I', duration=4.0, loop=0) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    for filename in filenames:
        os.remove(filename)

    print(f"GIF saved as {gif_filename}")


In [44]:

def make_superpose_df(df1, df2):
    os.makedirs('frames', exist_ok=True)
    folder_path = "gifs"
    os.makedirs(folder_path, exist_ok=True)
    
    filenames = []

    for i in range(len(df1)):
        frame_path = f'frames/frame_{i:03d}.png'
        filenames.append(frame_path)
        
        fig, ax = plt.subplots(figsize=(14, 12))
        
        vertex1 = df1.iloc[i]['vertex']
        u_old1 = df1.iloc[i]['u_old']
        u_old1 = [i/sum(u_old1) for i in u_old1]

        vertex2 = df2.iloc[i]['vertex']
        u_old2 = df2.iloc[i]['u_old']
        u_old2 = [i/sum(u_old2) for i in u_old2]
        
        ax.bar(range(vertex1), u_old1, color = 'blue') #find a way to possibly center this-> maybe ecenter then play with the width?
        ax.bar(range(vertex2), u_old2, color = 'red')
        ax.set_xlabel('vertex')
        ax.set_ylabel('Prob')
        ax.set_title(f"superposition of df1 and df2")

        ax.set_ylim(0, 1)
        ticks = np.arange(0, max(u_old1) + 0.01, 1)
        ax.set_yticks(ticks)
        
        fig.savefig(frame_path)
        plt.close(fig)
    
    filename = f"superposition between 10 and 15"
    gif_filename = f"{folder_path}/{filename}.gif"
    
    with imageio.get_writer(gif_filename, mode='I', duration=4.0, loop = 0) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)
    

    for filename in filenames:
        os.remove(filename)
    
    print(f"GIF saved as ")

In [ ]:
#from this point onwards, it's old jsut kept just in case as a backup

In [ ]:
#previous way of stopping the iterations
centers = None
u_old = None
matrix = matrix_uniform_attachment(15)
delta = 0.3
matri = diffusion_process(matrix, delta)
iterations_done = 0
last_four_value = []

tolerance = 1e-5
for _ in range(20):
    x,y = calculate_u_new(matrix, u_old = u_old, centers = centers)
    centers = y
    value = np.matmul(matri, np.array(x))
    u_old = value

    if len(last_four_value) < 4:
        last_four_value.append(value)
    elif len(last_four_value) >= 4:
        last_four_value.remove(last_four_value[0])
        last_four_value.append(value)


    iterations_done+=1
    if len(last_four_value) >= 4:
        diffs = [np.linalg.norm(last_four_value[i] - last_four_value[i - 1]) for i in range(1, 4)]
        if np.mean(diffs) < tolerance:
            break



In [ ]:
delta_range = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
matrix_sizes = list(range(5,14))

def boss_runner(delta_range, matrix_sizes, tolerance = 1e-12, model_type = "random_uni"):

    all_data = []
    
    for matrix_size in matrix_sizes:   
        iteration = []
        for delta in delta_range:
            for _ in range(10):
                local_it = []
                local_entropy = []
                u_new, iterr, entropy = main_guy(delta, model_type=model_type, matrix_size=matrix_size, tolerance=tolerance)
                local_it.append(iterr)
                local_entropy.append(entropy)
            iteration.append({'delta':delta ,
                              'vertex':matrix_size,
                            'iterration':np.mean(local_it),
                            'entropy': np.mean(local_entropy),
                             'u_old' : u_new})
    
        all_data.append(iteration)
        df = pd.DataFrame(iteration)
    
        plt.figure(figsize=(8, 5))
        sns.set(style="whitegrid")
        
        
        plt.semilogy(df['delta'], df['u_old'], marker='o', linestyle='-', color='b', 
                     label=f'Entropy (Matrix Size: {matrix_size}, Tolerance: {tolerance}, Model: {model_type})')
        
        
        plt.xlabel("(delta)", fontsize=12)
        plt.ylabel("Entropy (log scale)", fontsize=12)
        title = f"Entropy vs. Delta, Vertex count is {df['vertex'][0]}, and initial entropy is {round(np.log(df['vertex'][0]), 3)}"
        plt.title(title, fontsize=14)
        plt.legend()
        plt.grid(True, which="both", linestyle="--", linewidth=0.5)
        
        

        folder_path = "images"
        os.makedirs(folder_path, exist_ok=True)
        path = os.path.join(folder_path, f"plot_{title}.png")
        plt.savefig(path)
        plt.show()
    
    return all_data